<a href="https://kaggle.com/kernels/welcome?src=https://github.com/fastai/fastai/blob/master/nbs/examples/migrating_catalyst.ipynb"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#all_slow

In [ ]:
from fastai.vision.all import *

# Tutorial - Migrating from Catalyst

> Incrementally adding fastai goodness to your Catalyst training

## Catalyst code

We're going to use the MNIST training code from Catalyst's README (as at August 2020), converted to a module.

In [ ]:
from migrating_catalyst import *

To use it in fastai, we first convert the Catalyst dict into a `DataLoaders` object:

In [ ]:
data = DataLoaders(loaders['train'], loaders['valid']).cuda()

### Using callbacks

In the Catalyst code, a training loop is defined manually, which is where the input tensor is flattened. In fastai, there's no need to define your own training loop - you can insert your own code into any part of the training process by using a callback, which can even modify data, gradients, the loss function, or anything else in the training loop:

In [ ]:
@before_batch_cb
def cb(self, xb, yb): return (xb[0].view(xb[0].size(0), -1),),yb

The Catalyst example also modifies the training loop to add metrics, but you can pass these directly to your `Learner` in fastai:

In [ ]:
metrics=[accuracy,top_k_accuracy]
learn = Learner(data, model, loss_func=F.cross_entropy, opt_func=Adam,
                metrics=metrics, cbs=cb)

You can now fit your model. fastai supports many schedulers. We recommend using 1cycle:

In [ ]:
learn.fit_one_cycle(1, 0.02)

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,0.230145,0.292590,0.924700,0.995000,00:13


As you can see, migrating from Catalyst allowed us to replace 17 lines of code (in `CustomRunner`) with just 3 lines, and doesn't require you to change any of your existing data pipelines, optimizers, loss functions, models, etc. Once you've made this change, you can then benefit from fastai's rich set of callbacks, transforms, visualizations, and so forth.

Note that fastai is very different from Catalyst, in that it is much more than just a training loop (although we're only using the training loop in this example) - it is a complete framework including GPU-accelerated transformations, end-to-end inference, integrated applications for vision, text, tabular, and collaborative filtering, and so forth. You can use any part of the framework on its own, or combine them together, as described in the [fastai paper](https://arxiv.org/abs/2002.04688).

### Changing the model

Instead of using callbacks, in this case you can also simply change the model. Here we pull the `view()` out of the training loop, and into the model, using fastai's `Flatten` layer:

In [ ]:
model = nn.Sequential(
    Flatten(),
    torch.nn.Linear(28 * 28, 10))

We can now create a `Learner` and train without using any callbacks:

In [ ]:
learn = Learner(data, model, loss_func=F.cross_entropy, opt_func=Adam, metrics=metrics)
learn.fit_one_cycle(1, 0.02)

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,0.230051,0.292577,0.924800,0.995100,00:11
